In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

input_shape = x_train.shape[1:]

Custom layers

In [3]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [4]:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

In [5]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer,
])

model.compile(loss='mse', optimizer='sgd')

model.fit(x_train_scaled, y_train, epochs=5, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/5
363/363 [==============================] - 0s 740us/step - loss: 0.6499 - val_loss: 0.4175
Epoch 2/5
363/363 [==============================] - 0s 602us/step - loss: 0.4631 - val_loss: 0.3865
Epoch 3/5
363/363 [==============================] - 0s 622us/step - loss: 0.3984 - val_loss: 0.3760
Epoch 4/5
363/363 [==============================] - 0s 599us/step - loss: 0.4053 - val_loss: 0.3592
Epoch 5/5
162/162 [==============================] - 0s 404us/step - loss: 0.5237


0.5236707329750061

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
_________________________________________________________________
lambda (Lambda)              (None, 1)                 0         
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [7]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=[batch_input_shape[-1], self.units], 
                                      initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', 
                                    shape=[self.units], 
                                    initializer='zeros')
        super().build(batch_input_shape)
        
    def call(self, x):
        return self.activation(x @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units, 
                'activation': keras.activations.serialize(self.activation)}

In [8]:
model = keras.models.Sequential([
    MyDense(30, activation='relu', input_shape=input_shape),
    MyDense(1),
])

model.compile(loss='mse', optimizer='nadam')

model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 823us/step - loss: 1.5019 - val_loss: 1.2678
Epoch 2/2
162/162 [==============================] - 0s 404us/step - loss: 0.4758


0.47581905126571655

In [9]:
model.save('model_with_custom_layer.h5')

In [10]:
model = keras.models.load_model('model_with_custom_layer.h5', 
                                custom_objects={'MyDense': MyDense})

In [11]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, x):
        x1, x2 = x
        return x1 + x2, x1 * x2
    
    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [12]:
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

In [13]:
def split_data(data):
    columns_count = data.shape[-1]
    half = columns_count // 2
    return data[:, :half], data[:, half:]

In [14]:
x_train_scaled_a, x_train_scaled_b = split_data(x_train_scaled)
x_valid_scaled_a, x_valid_scaled_b = split_data(x_valid_scaled)
x_test_scaled_a, x_test_scaled_b = split_data(x_test_scaled)

x_train_scaled_a.shape, x_train_scaled_b.shape

((11610, 4), (11610, 4))

In [15]:
input_a = keras.layers.Input(shape=x_train_scaled_a.shape[-1])
input_b = keras.layers.Input(shape=x_train_scaled_b.shape[-1])
hidden_a, hidden_b = MyMultiLayer()((input_a, input_b))
hidden_a = keras.layers.Dense(30, activation='selu')(hidden_a)
hidden_b = keras.layers.Dense(30, activation='selu')(hidden_b)
concat = keras.layers.Concatenate()((hidden_a, hidden_b))
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_a, input_b], outputs=[output])

In [16]:
model.compile(loss='mse', optimizer='nadam')

model.fit((x_train_scaled_a, x_train_scaled_b), y_train, epochs=2, 
          validation_data=((x_valid_scaled_a, x_valid_scaled_b), y_valid))

model.evaluate((x_test_scaled_a, x_test_scaled_b), y_test)

Epoch 1/2
363/363 [==============================] - 1s 854us/step - loss: 1.8849 - val_loss: 4.0209
Epoch 2/2
162/162 [==============================] - 0s 429us/step - loss: 1.0414


1.0413503646850586

In [17]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, x, training=None):
        if training:
            noise = tf.random.normal(tf.shape(x), stddev=self.stddev)
            return x + noise
        else:
            return x
        
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [18]:
model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation='selu'),
    keras.layers.Dense(1),
])

In [19]:
model.compile(loss='mse', optimizer='nadam')

model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 760us/step - loss: 2.4692 - val_loss: 0.9278
Epoch 2/2
162/162 [==============================] - 0s 416us/step - loss: 0.7787


0.7787245512008667

Custom models

In [20]:
x_new_scaled = x_test_scaled

In [21]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation='elu', 
                                          kernel_initializer='he_normal')
                       for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [22]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation='elu', 
                                          kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
        
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [23]:
model = ResidualRegressor(1)
model.compile(loss='mse', optimizer='nadam')
history = model.fit(x_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 1s 989us/step - loss: 8.1168
Epoch 2/5
363/363 [==============================] - 0s 884us/step - loss: 1.7072
Epoch 3/5
363/363 [==============================] - 0s 829us/step - loss: 1.3773
Epoch 4/5
363/363 [==============================] - 0s 854us/step - loss: 1.2679
Epoch 5/5
363/363 [==============================] - 0s 890us/step - loss: 0.9363


In [24]:
score = model.evaluate(x_test_scaled, y_test)
score

162/162 [==============================] - 0s 528us/step - loss: 0.6157


0.6156613826751709

In [25]:
y_pred = model.predict(x_new_scaled)
y_pred

array([[0.07981298],
       [1.8383291 ],
       [4.5888257 ],
       ...,
       [1.5495952 ],
       [2.7323132 ],
       [4.211289  ]], dtype=float32)

In [26]:
model.save('custom_model.ckpt')

INFO:tensorflow:Assets written to: custom_model.ckpt\assets


INFO:tensorflow:Assets written to: custom_model.ckpt\assets


In [27]:
model = keras.models.load_model('custom_model.ckpt')

In [28]:
history = model.fit(x_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 1s 887us/step - loss: 0.7770
Epoch 2/5
363/363 [==============================] - 0s 851us/step - loss: 0.9922
Epoch 3/5
363/363 [==============================] - 0s 847us/step - loss: 0.4034
Epoch 4/5
363/363 [==============================] - 0s 845us/step - loss: 0.7274
Epoch 5/5
363/363 [==============================] - 0s 856us/step - loss: 0.8868


In [29]:
block1 = ResidualBlock(2, 30)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='elu', 
                       kernel_initializer='he_normal'),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1),
])

model.compile(loss='mse', optimizer='nadam')
history = model.fit(x_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 1s 653us/step - loss: 1.3901
Epoch 2/5
363/363 [==============================] - 0s 657us/step - loss: 0.6113
Epoch 3/5
363/363 [==============================] - 0s 644us/step - loss: 0.4894
Epoch 4/5
363/363 [==============================] - 0s 652us/step - loss: 0.6447
Epoch 5/5
363/363 [==============================] - 0s 652us/step - loss: 0.4279


In [30]:
score = model.evaluate(x_test_scaled, y_test)
score

162/162 [==============================] - 0s 441us/step - loss: 0.4446


0.4446283280849457

In [31]:
y_pred = model.predict(x_new_scaled)
y_pred

array([[0.6536274],
       [1.8239018],
       [3.635159 ],
       ...,
       [1.3967994],
       [2.6738923],
       [4.203689 ]], dtype=float32)

In [32]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation='selu', 
                                          kernel_initializer='lecun_normal')
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = keras.layers.Dense(8)
        self.reconstruction_mean = keras.metrics.Mean(name='reconstruction_error')
        
    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        self.recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        
        if training:
            result = self.reconstruction(recon_loss)
            self.add_metric(result)
        return self.out(Z)
    
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred, regularization_losses=[self.recon_loss])
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {m.name: m.result() for m in self.metrics}    

In [33]:
model = ReconstructingRegressor(1)
model.compile(loss='mse', optimizer='nadam')
history = model.fit(x_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 699us/step - loss: 0.7098 - reconstruction_error: 0.0000e+00
Epoch 2/2
363/363 [==============================] - 0s 689us/step - loss: 0.4665 - reconstruction_error: 0.0000e+00


In [34]:
y_pred = model.predict(x_test_scaled)
y_pred

array([[0.79058826],
       [1.8234769 ],
       [3.8583977 ],
       ...,
       [1.6587366 ],
       [2.3147085 ],
       [3.9397335 ]], dtype=float32)